<a href="https://colab.research.google.com/github/ekgren/workshop/blob/main/Day2/QA_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU, Transformers, Bert m.m.



In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

In [ ]:
import datasets
import transformers
import torch
import copy
import tqdm

# Exempel KB-Bert med ordmaskning

När man förtränar Bert modeller så lär de sig språklig statistik genom att se massa text, maska ord och gissa vilket ord som bör vara på en den maskerade platsen. I slutändan är det nästan aldrig gissa ord som modellerna används till utan man anpassar (finetunear) dem till en annan uppgift. Men för att illustrera hur den förtränade modellen fungerar så gör vi en maskningsuppgift.

Vi börjar med att ladda KB-Bert och dess tokeniserare.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
model = transformers.BertForMaskedLM.from_pretrained('KB/bert-base-swedish-cased')
print(type(model))

Vi hittar på en exempelmening.

In [ ]:
example = 'Hej och välkommen till Trafikverket! Myndigheten för dig som gillar vägar, bilar och tåg.'
example

Bert är tränad med speciella ord i början och slutet av meningar, [CLS] och [SEP]. Modellen förutsätter att de är med när du stoppar in en mening. Om du skapar en batch med en huggingface tokeniserare görs detta automatiskt av tokeniseraren men i det här exemplet lägger vi till dem manuellt till exempelmeningen. 

In [ ]:
example_preprocessed = f'[CLS] {example} [SEP]'
example_preprocessed

Nu har vi vår mening i textform och nästa steg är att dela upp den i tokens med vår tokeniserare.

In [ ]:
tokens = tokenizer.tokenize(example_preprocessed)
print(tokens)

Sedan gör vi om våra tokens till index som modellen använder för att ta fram en sifferrepresentation av de tokens som går in i modellen.

In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(indexed_tokens)

Nu till själva uppgiften som vi skall utföra med modellen. Vi väljer ut ett ord som ligger på plats 5 med nollindexering, "Trafikverket", och ersätter det med en [MASK] token. Detta för att sedan låta modellen givet resten av meningen gissa vilket ord som passar bäst in istället för [MASK].

In [ ]:
masked_index = 5
tokens[masked_index] = '[MASK]'
print(tokens)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print("\nToken index:", indexed_tokens)

Sedan matar vi in vår exempelmening i KB-Bert

In [ ]:
# Transformers använder dropout under träning. Om man vill säga åt modellen
# att stänga av dropout använder man model.eval(). Detta bör man göra när man
# applicerar model, som här.

_ = model.eval()

# För att spara på datorkraft säger åt torch att inte räkna fram gradienter.
with torch.no_grad():
    outputs = model(torch.tensor([indexed_tokens]))

predictions = outputs[0]
print(predictions.shape)

In [ ]:
predicted_index = torch.topk(predictions[0, masked_index], k=5)
print(predicted_index.indices)

In [ ]:
predicted_token = tokenizer.convert_ids_to_tokens(predicted_index.indices)
print(predicted_token)

# Ladda modeller och tokeniserare

Vi börjar med att ladda ner modell och tokeniserare från huggingface.

Både modell och tokeniserare är tränad på Kungliga Bibliotekets data, och har använts i många svenska NLP-applikationer.


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
model = transformers.AutoModel.from_pretrained('KB/bert-base-swedish-cased')

# Finetuning: Semantisk likhet mellan *meningar*

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
kb_bert = transformers.AutoModel.from_pretrained('KB/bert-base-swedish-cased')

## Data

För detta exempel använder vi STS-B-datasetet maskinöversatt till svenska. 

In [ ]:
dataset = datasets.load_dataset('stsb_mt_sv')
train_ds = dataset['train']
test_ds = dataset['test']
eval_ds = dataset['validation']

Det är alltid bra att inspektera datan så vi tittar på några exempel från datasetet för att bilda oss en uppfattning om uppgiften.

In [ ]:
for ix in range(10):
    print(dataset['train'][ix])

I utskriften ovanför ser vi att datan är i json format eller snarare en python dictionary. Vi skriver lite kod för att formatera den så att den är mer lättläst.  

Datan består av tre delar: 'sentence1', 'sentence2' samt 'score'

In [ ]:
for ix in range(10):
  s1 = train_ds[ix]['sentence1']
  s2 = train_ds[ix]['sentence2']
  score = train_ds[ix]['score']
  print('{:.2f}: {:^40} | {:^40}'.format(score, s1, s2))

## Dataloaders

För att kunna mata nätverket med data så behöver vi göra dataloaders. Dataloaders samlar ihop datan till rätt format samt batchar den.


In [ ]:
def encode(*texts):
    # Den här funktionen tar in en lista med texter
    # och returnerar tokeniserad data i pytorch format.
    assert 1 <= len(texts) <= 2
    return tokenizer(*texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

def collate_paired(rows):
    # Den här funktionen parar ihop två meningar med en [SEP] token mellan dem.
    # Detta för att kunna fineatunea på STS.
    s1s = [row['sentence1'] for row in rows]
    s2s = [row['sentence2'] for row in rows]
    scores = torch.tensor([row['score'] for row in rows])
    return encode(s1s, s2s), scores

Nedan förbereder vi "dataloaders" som är torchabstraktioner för att hantera data. Dessa hjälper till med att ladda datan parallelt (med multiprocessing),
blanda datan, samt stycka upp den i mindre "batches": Detta har visat sig ge både bättre resultat när man tränar modeller, och är nödvändigt om man har stora modeller och många träningsdatapunkter.

In [ ]:
def make_dataloaders(collater, train_batch_size=8, test_batch_size=None):

    if test_batch_size is None:
        test_batch_size = 2 * train_batch_size

    train_dl = torch.utils.data.DataLoader(  
        train_ds,
        collate_fn=collater,
        shuffle=True,
        batch_size=train_batch_size,
        pin_memory=True,
    )

    test_dl = torch.utils.data.DataLoader(  
        test_ds,
        collate_fn=collater,
        shuffle=False,
        batch_size=test_batch_size,
        pin_memory=True,
    )

    eval_dl = torch.utils.data.DataLoader(  
        eval_ds,
        collate_fn=collater,
        shuffle=False,
        batch_size=test_batch_size,
        pin_memory=True,
    )

    return train_dl, test_dl, eval_dl


## En enkel sts-modell

In [ ]:
class PairedModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = copy.deepcopy(kb_bert)
        self.linear = torch.nn.Linear(768, 1)

    def forward(self, data):
        return self.linear(self.model(**data)['pooler_output']).squeeze(-1)

## Hjälpfunktioner

Här definierar vi hjälpfunktioner för att räkna ut medelvärden, och för att 
flytta data till GPUer

In [ ]:

class ExpMean(object):
    """
    Class for calculating an online exponential mean
    """
    def __init__(self, mean=None, alpha=0.1):
        self.mean = None
        self.alpha = alpha

    def __iadd__(self, other):
        if self.mean is None:
            self.mean = other
        else:
            self.mean += (other - self.mean) * self.alpha
        return self

class WelfordMean(object):                                                    
    """                                                                       
    Class for calculating an online weighted mean                             
    """                                                                       
    def __init__(self, mean=0.0, weight=0.0):                                 
        self.mean = mean                                                      
        self.weight = weight                                                  
                                                                            
    def __iadd__(self, other):                                                
    if type(other) is WelfordMean:                                        
        self.weight += other.weight                                       
        self.mean += (other.mean - self.mean) * other.weight / self.weight
    else:                                                                 
        self.weight += 1                                                  
        self.mean += (other - self.mean) / self.weight                    
    return self

def to_cuda(xs):
    if type(xs) is list:
        return [to_cuda(x) for x in xs]
    elif type(xs) is dict:
        return {k: to_cuda(v) for k, v in xs.items()}
    elif type(xs) is torch.Tensor:
        return xs.cuda()
    else:
        raise ValueError

### Evaluering

In [ ]:

@torch.no_grad()
def eval_model(model, dataloader):
    # Skapa en tqdm-progress bar
    batches = tqdm.tqdm(dataloader)

    # Initialisera statistik
    acc_loss = WelfordMean()
    for batch, score in batches:
        # Stoppa datan på rätt "plats" (gpu)
        batch = to_cuda(batch)
        score = to_cuda(score)

        # Beräkna sifferpoängen enligt modellen
        prediction = model(batch)
        loss = torch.nn.functional.mse_loss(prediction, score)
        acc_loss += WelfordMean(loss.item(), len(batch))

    return acc_loss.mean

### Träning

In [ ]:
def train_epoch(model, optimizer, dataloader):
    # Skapa en tqdm-progress bar
    batches = tqdm.tqdm(dataloader)

    # Initialisera statistik
    acc_loss = ExpMean()
    for ix, (batch, score) in enumerate(batches):
        # Stoppa datan på rätt "plats" (gpu)
        batch = to_cuda(batch)
        score = to_cuda(score)

        # Nollställ parametrarnas gradienter
        optimizer.zero_grad()

        # Beräkna sifferpoängen enligt modellen
        prediction = model(batch)

        # Beräkna lossen (MSE)
        loss = torch.nn.functional.mse_loss(prediction, score)

        # Beräkna gradienten av lossen med avseende på modellens parametrar.
        loss.backward()

        # Uppdatera parametrarna
        optimizer.step()

        acc_loss += loss.item()

        batches.set_description('loss {:.2f}'.format(acc_loss.mean))

## Träningsloopen

Här tränar vi modellen! 

notera 

In [ ]:
model = PairedModel().cuda()
train_dl, test_dl, eval_dl = make_dataloaders(collate_dual)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
EPOCHS = 1
for epoch in range(EPOCHS):
    # Utvärdera modellen
    print('\nafter {} epochs, loss {:.2f}'.format(epoch, eval_model(model, test_dl)))

    # Träna modellen på all träningsdata
    train_epoch(model, optimizer, train_dl)

#Utvärdera modellen
print('\nafter {} epochs, loss {:.2f}'.format(EPOCHS, eval_model(model, test_dl)))    

In [ ]:
batch, score = next(iter(train_dl))

with torch.no_grad():
    for predicted_score, text, true_score in zip(
        model(to_cuda(batch)),
        [tokenizer.decode(ids[ids != 0]) for ids in batch['input_ids']],
        score):
    print(text)

    print('prediction: {:.2f}    truth: {:.2f}'.format(predicted_score.item(), true_score.item()))

In [ ]:
text1 = 'Hur öppnar jag outlook?'
text2 = 'Hur öppnar jag min mail-klient?'

with torch.no_grad():
    print(model({k:v.cuda() for k, v in encode([text1], [text2]).items()}))

# Namnigenkänning (Named Entity Recognition)

Exempel: Namnigenkänning

Kort förklaring av namnigenkänning

BERT base fine-tuned for Swedish NER. This model is fine-tuned on the SUC 3.0 dataset.

Entity types used are TME for time, PRS for personal names, LOC for locations, EVN for events and ORG for organisations.

In [ ]:
from transformers import pipeline

nlp = pipeline('ner', model='KB/bert-base-swedish-cased-ner', tokenizer='KB/bert-base-swedish-cased-ner')

In [ ]:
nlp('Hej jag heter Bertil och jobbar på Trafikverket i Målilla och jag vill byta lösenord.')